In [12]:
]activate ~/.config/julia/projects/oft

 Activating environment at `~/.config/julia/projects/oft/Project.toml`


## Babylonian sqrt
> Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.
 
This notebook is to make the Babylonian square root algorithm less mytical, if ever it seems. Recall our questions from `Automatic Differentiation in 10 Minutes.ipynb`:

> **(?1)** Questions about Babylonian sqrt.
>
> - Why this Babylonian iteration converges to sqrt(x)?
> - Does it matter to start with `t=1`?

**(R1)** Actually, it was answered in the original YouTube video and is quite straight-forward: This Babylonian iteration converges because at every iteration, be it `t > √x` or `t <= √x`,
`x` will always lie in between `t` and `x/t`. That is, among `t` and `x/t`, one is greater
than `x`, the other less than `x`. We then take the midpoint of the two, and proceed to the
next iteration. Convergence is expected.

In [13]:
function babylonian(x; t=1, N=10)
    for i = 1:N; t = (t + x/t)/2  end    
    t
end  

babylonian (generic function with 1 method)

Check that it works:

In [14]:
α = π
babylonian(α), √α    

(1.7724538509055159, 1.7724538509055159)

In [15]:
babylonian(α, t=100), babylonian(α, t=9999), babylonian(α, t=-100) 

(1.7724538509055165, 9.871657258137894, -1.7724538509055165)

In [16]:
for t₀ in [10,100,1000,10_000]
    println(babylonian(α, t=t₀))
end

1.7724538509055159
1.7724538509055165
1.8690126833827456
9.872623166568108


In [17]:
babylonian(α, t=10_000), babylonian(α, t=10_000, N=50), √α

(9.872623166568108, 1.7724538509055159, 1.7724538509055159)

So, according to the above experiments, the starting value of `t` matters a little bit, in the way that
- if the starting value is **positive**, then **the greater** it (i.e. `t0` above) is, **the more** iterations (i.e. `N`) it requires **to reach the same level of convergence**.
- if the starting value is **negative**, then the result will converge to $-\sqrt{x}$.

**(?1)** Can this Babylonian function have a recursive definition? Tail-recursive?

In [18]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

LoadError: UndefVarError: Babylonian not defined

In [ ]:
# Pkg.add(plots)
# Pkg.add(plotly)
using Plots
plotly()
#gr()
#pyplot()

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [10]:
## Warning first plots load packages, takes time
i = 0:.01:49

# Note the diff i's and their scopes
#plot([x->Babylonian(x,N=i) for i=1:5],i,label=["Iteration $j" for i=1:1,j=1:5])
plot([x->Babylonian(x,N=i) for i=1:5],i,label=["Iteration $j" for j=1:5])

plot!(sqrt,i,c="black",label="sqrt",
      title="Those Babylonians really knew how to √")

LoadError: UndefVarError: plot not defined

**(?1)**
Why this Babylonian iteration converges to `sqrt(x)`?

## ...and now the derivative, almost by magic

Eight lines of Julia!
- No mention of $\frac{1}{2} x^{-\frac{1}{2}}$.
- `D` for "**dual number**", invented by the famous algebraist Clifford in 1873.

In [26]:
struct D <: Number  # D is a function-derivative pair
    f::Tuple{Float64,Float64}
end

Sum Rule: (x+y)' = x' + y' <br>
Quotient Rule: (x/y)' = (yx'-xy') / y^2

In [40]:
import Base: +, /, convert, promote_rule
+(x::D, y::D) = D(x.f .+ y.f)
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D}, x::Real) = D((x,zero(x)))  # derivative of a constant
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 148 methods)

For any given `D`, its `f` will store
- `f[1]`: some function $u$
- `f[2]`: the derivative $u'$

In [34]:
Type(D)

LoadError: MethodError: no method matching Type(::Type{D})

In [31]:
Type(Number)

LoadError: MethodError: no method matching Type(::Type{Number})

**curly brackets** `Type{}`, not **parentheses** `Type()`.

In [32]:
Type{D}

Type{D}

In [33]:
Type{Number}

Type{Number}

What is `zero(x)` when `x` is a `Real`?

In [36]:
x = 2.71828

2.71828

In [37]:
zero(x)

0.0

The same algorithm with no rewrite at all computes properly
the derivative as the check shows.

In [41]:
x=49; Babylonian(D((x,1))), (√x,.5/√x)

(D((7.0, 0.07142857142857142)), (7.0, 0.07142857142857142))

In [42]:
x=π; Babylonian(D((x,1))), (√x,.5/√x)

(D((1.7724538509055159, 0.28209479177387814)), (1.7724538509055159, 0.28209479177387814))

In [43]:
aaa = "aaa"
bbb = "bbb"

"bbb"

**Unlike `Pluto`**, in Jupyter notebook, to write a multi-line code cell we **don't need** to write `begin ... end`

## It just works!

How does it work?  We will explain in a moment.  Right now marvel that it does.  Note we did not
import any autodiff package.  Everything is just basic vanilla Julia.

## The assembler

Most folks don't read assembler, but one can see that it is short.
The shortness is a clue that suggests speed!

In [44]:
@inline function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  
@code_native(Babylonian(D((2,1))))

	.text
; ┌ @ In[44]:1 within `Babylonian'
	movq	%rdi, %rax
; │ @ In[44]:2 within `Babylonian'
; │┌ @ In[44]:2 within `#Babylonian#32'
; ││┌ @ promotion.jl:311 within `+' @ In[40]:2
; │││┌ @ broadcast.jl:837 within `materialize'
; ││││┌ @ broadcast.jl:1046 within `copy'
; │││││┌ @ ntuple.jl:42 within `ntuple'
; ││││││┌ @ broadcast.jl:1046 within `#19'
; │││││││┌ @ broadcast.jl:621 within `_broadcast_getindex'
; ││││││││┌ @ broadcast.jl:648 within `_broadcast_getindex_evalf'
; │││││││││┌ @ float.jl:401 within `+'
	vmovsd	(%rsi), %xmm1           # xmm1 = mem[0],zero
	movabsq	$.rodata.cst8, %rcx
	vaddsd	(%rcx), %xmm1, %xmm4
	vmovsd	8(%rsi), %xmm2          # xmm2 = mem[0],zero
	vxorpd	%xmm8, %xmm8, %xmm8
	vaddsd	%xmm8, %xmm2, %xmm5
	movabsq	$140175471296264, %rcx  # imm = 0x7F7D252C3F08
	vmovsd	(%rcx), %xmm3           # xmm3 = mem[0],zero
; ││└└└└└└└└
; ││┌ @ promotion.jl:314 within `/' @ In[40]:3 @ float.jl:407
	vmulsd	%xmm3, %xmm4, %xmm6
; │││ @ promotion.jl:314 within `/' @ In[40]:3
; ││

## Symbolically

We haven't yet explained how it works, but it may be of some value to understand that the below is mathematically
equivalent, though not what the computation is doing.

Notice in the below that Babylonian works on SymPy symbols.

Note: Python and Julia are good friends.  It's not a competition!  Watch how nicely we can use the same code now with SymPy.

In [47]:
using Pkg

In [49]:
Pkg.add("SymPy")
using SymPy                

  Resolving package versions...
  Installed PyCall ─ v1.92.1
  Installed SymPy ── v1.0.32
Updating `~/.julia/environments/v1.5/Project.toml`
  [24249f21] + SymPy v1.0.32
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [438e738f] + PyCall v1.92.1
  [24249f21] + SymPy v1.0.32
   Building PyCall → `~/.julia/packages/PyCall/BcTLp/deps/build.log`
┌ Info: Precompiling SymPy [24249f21-da20-56a4-8eb1-6a02cf4ae2e6]
└ @ Base loading.jl:1278


LoadError: InitError: PyError (PyImport_ImportModule

The Python package sympy could not be imported by pyimport. Usually this means
that you did not install sympy in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/home/phunc20/.julia/conda/3/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the sympy module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run P

In [ ]:
x = symbols("x")
display("Iterations as a function of x")
for k = 1:5
 display( simplify(Babylonian(x,N=k)))
end

display("Derivatives as a function of x")
for k = 1:5
 display(simplify(diff(simplify(Babylonian(x,N=k)),x)))
end

The code is computing answers mathematically equivalent to the functions above, but not symbolically, numerically. 

## How autodiff is getting the answer
Let us by hand take the "derivative" of the Babylonian iteration with respect to x. Specifically t′=dt/dx.  This is the old fashioned way of a human rewriting code.

In [50]:
function dBabylonian(x; N = 10) 
    t = (1+x)/2
    t′ = 1/2
    for i = 1:N;  
        t = (t+x/t)/2; 
        t′= (t′+(t-x*t′)/t^2)/2; 
    end    
    t′

end  

dBabylonian (generic function with 1 method)

See this rewritten code gets the right answer.  So the trick is for the computer system to do it for you, and without any loss of speed or convenience.

In [51]:
x = π; dBabylonian(x), .5/√x

(0.2820947917738782, 0.28209479177387814)

What just happened?  Answer: We created an iteration by hand for t′ given our iteration for t. Then we ran the iteration alongside the iteration for t.

In [ ]:
Babylonian(D((x,1)))

How did this work?  It created the same derivative iteration that we did by hand, using very general rules that are set once and need not be written by hand.

Important:: The derivative is substituted before the JIT compiler, and thus efficient compiled code is executed.

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.) 

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a+c) \pm (b+d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/d^2 \epsilon $


In [ ]:
Base.show(io::IO,x::D) = print(io,x.f[1]," + ",x.f[2]," ϵ")

In [ ]:
# Add the last two rules
import Base: -,*
-(x::D, y::D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))

In [ ]:
D((1,0))

In [ ]:
D((0,1))^2

In [ ]:
D((2,1)) ^2

In [ ]:
ϵ = D((0,1))
@code_native(ϵ^2)

In [ ]:
ϵ * ϵ 

In [ ]:
ϵ^2

In [ ]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

In [ ]:
(1+ϵ)^5 ## Note this just works (we didn't train powers)!!

## Generalization to arbitrary roots

In [ ]:
function nthroot(x, n=2; t=1, N = 10) 
    for i = 1:N;   t += (x/t^(n-1)-t)/n; end   
    t
end  

In [ ]:
nthroot(2,3), ∛2 # take a cube root

In [ ]:
nthroot(2+ϵ,3)

In [ ]:
nthroot(7,12), 7^(1/12)

In [ ]:
x = 2.0
nthroot( x+ϵ,3), ∛x, 1/x^(2/3)/3

## Forward Diff
Now that you understand it, you can use the official package

In [ ]:
Pkg.add("ForwardDiff")

In [ ]:
using ForwardDiff

In [ ]:
ForwardDiff.derivative(sqrt, 2)

In [ ]:
ForwardDiff.derivative(Babylonian, 2)

In [ ]:
@which ForwardDiff.derivative(sqrt, 2)

## Close Look at Convergence with big floats
the -log10 gives the number of correct digits.  Watch the quadratic convergence right before your eyes.

In [ ]:
setprecision(3000)
round.(Float64.(log10.([Babylonian(BigFloat(2),N=k) for k=1:10] - √BigFloat(2))),3)

In [ ]:
struct D1{T} <: Number  # D is a function-derivative pair
    f::Tuple{T,T}
end

In [ ]:
z = D((2.0,1.0))
z1 = D1((BigFloat(2.0),BigFloat(1.0)))

In [ ]:
import Base: +, /, convert, promote_rule
+(x::D1, y::D1) = D1(x.f .+ y.f)
/(x::D1, y::D1) = D1((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D1{T}}, x::Real) where {T} = D1((convert(T, x), zero(T)))
promote_rule(::Type{D1{T}}, ::Type{S}) where {T,S<:Number} = D1{promote_type(T,S)}

In [ ]:
A = randn(3,3)

In [ ]:
x = randn(3)

In [ ]:
ForwardDiff.gradient(x->x'A*x,x)

In [ ]:
(A+A')*x

In [ ]:
n = 4
Strang = SymTridiagonal(2*ones(n),-ones(n-1))

##  But wait there's more!

Many packages need to be taught how to compute autodiffs of matrix factorications such as the svd or lu.  Julia will "just do it," no
teaching necessary for reasons such as the above.  This is illustrated in another notebook, not included here.